## How much money can a director lose before they disappear in the industry?

### How many box office bombs can an established director endure?

In [86]:
# 20200726 JLCY

import pandas as pd
import numpy as np
import math

xl = pd.ExcelFile('directors_data.xlsx')
print('number of directors:', len(xl.sheet_names))

number of directors: 50


In [87]:
append_data = []

for sheet_name in xl.sheet_names: 

    xl_file = pd.read_excel('directors_data.xlsx', sheet_name=sheet_name)

    # combine 1st 2 rows as column names
    combine_col = []

    for ind, col in enumerate(xl_file.columns): 

        if isinstance(xl_file.iloc[0, ind], str): 
            combine_col.append(col + ' ' + xl_file.iloc[0, ind])
        else: 
            combine_col.append(col)

    combine_col = ['Opening Theaters' if i == 'Opening.1 Theaters' else i for i in combine_col]

    xl_file.columns = combine_col

    df = xl_file.iloc[1:-3] # drop 1st row and last 3 summary rows
    df.reset_index(inplace=True)
    df = df.iloc[:, 1:] # drop 1st duplicating column

    # populate data for multiple roles in the same movie
    nan_col = [col for col in df.columns if col not in ['Role']]

    for i in df.index: 

        if type(df['Title'][i]) is float: 

            if math.isnan(df['Title'][i]) and isinstance(df['Role'][i], str): 

                for col in nan_col: 
                    df[col][i] = df[col][i-1] # set data to the row above

    df['Person'] = sheet_name
    
    append_data.append(df)
    
combine_df = pd.concat(append_data)

In [88]:
len(combine_df['Person'].unique())

50

In [89]:
combine_df

,Release Date,Title,Role,Distributor,Genre,Production Budget,Opening Weekend,Opening Theaters,Maximum Theaters,Theatrical Engagements,Domestic Box Office,International Box Office,Worldwide Box Office,Person
0,2021-06-04 00:00:00,Micronauts,Producer,Paramount Pictures,Adventure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,J.J. Abrams
1,2019-12-20 00:00:00,Star Wars: The Rise of Skyw…,Director,Walt Disney,Adventure,275000000,177383864,4406,4406,29704,515202542,558267058,1073469600,J.J. Abrams
2,2019-12-20 00:00:00,Star Wars: The Rise of Skyw…,Screenwriter,Walt Disney,Adventure,275000000,177383864,4406,4406,29704,515202542,558267058,1073469600,J.J. Abrams
3,2019-12-20 00:00:00,Star Wars: The Rise of Skyw…,Producer,Walt Disney,Adventure,275000000,177383864,4406,4406,29704,515202542,558267058,1073469600,J.J. Abrams
4,2018-11-09 00:00:00,Overlord,Producer,Paramount Pictures,Action,38000000,10202108,2859,2859,8038,21704844,17473345,39178189,J.J. Abrams
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,2008-08-29 00:00:00,Disaster Movie,Director,Lionsgate,Comedy,20000000,5836973,2642,2642,8771,14190901,22529851,36720752,Aaron Seltzer
11,2008-01-25 00:00:00,Meet the Spartans,Director,20th Century Fox,Comedy,30000000,18505530,2605,2643,10672,38233676,46413155,84646831,Aaron Seltzer
12,2007-01-26 00:00:00,Epic Movie,Director,20th Century Fox,Comedy,20000000,18612544,2801,2840,11371,39739367,47119211,86858578,Aaron Seltzer
13,2006-02-17 00:00:00,Date Movie,Director,20th Century Fox,Comedy,20000000,19076261,2896,2898,13117,48548426,36597739,85146165,Aaron Seltzer


In [28]:
df = df.loc[df['Release Date'].dt.year < 2020] # 2020 is an outlier...

df['Profit'] = df['Worldwide Box Office'] - df['Production Budget']

In [29]:
len(df.loc[(df['Role'] == 'Director') & df['Profit'] > 0])

6